In [1]:
%matplotlib inline

%reload_ext autoreload
%autoreload 2

import keras

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
import json
from random import shuffle
import csv

import tensorflow as tf

def get_session():
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.compat.v1.Session(config=config)

tf.compat.v1.keras.backend.set_session(get_session())

Using TensorFlow backend.


In [2]:
model_path = 'data/snapshots_penguin/final_model_penguin.h5'

print(model_path)

model = models.load_model(model_path, backbone_name='resnet50')

print(model.summary())

labels_to_names = {0: 'penguin'}

data/snapshots_penguin/final_model_penguin.h5
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[ -7.601329 ,  -5.4425516,   7.601329 ,   5.4425516],
       [-12.007074 ,  -8.597065 ,  12.007074 ,   8.597065 ],
       [-19.362589 , -13.863615 ,  19.362589 ,  13.863615 ],
       [ -6.432    ,  -6.432    ,   6.432    ,   6.432    ],
       [-10.16     , -10.16     ,  10.16     ,  10.16     ],
       [-16.384    , -16.384    ,  16.384    ,  16.384    ],
       [ -5.4438148,  -7.5995655,   5.4438148,   7.5995655],
       [ -8.59906  , -12.004289 ,   8.59906  ,  12.004289 ],
       [-13.866832 , -19.358097 ,  13.866832 ,  19.358097 ]],
      dtype=float32)> anchors
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-15.202658 , -10.885103 ,  15.202658 ,  10.885103 ],
       [-24.014149 , -17.19413  ,  24.014149 ,  17.19413  ],
       [-38.725178 , -27.72723  ,  38.725178 ,  27.72723  ],
       [-12.864    , -12.864    ,  12.864    ,  12.

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
cwd = os.chdir('data/mosaics/penguin/labeled_tiles_penguin')

In [4]:
print (os.getcwd())

/host/seabirdNET/data/mosaics/penguin/labeled_tiles_penguin


In [5]:
annotations_file = 'penguin_test_annotations_final.csv'
cwd = os.getcwd()

image_names = []

with open (os.path.join(cwd, annotations_file)) as csvfile:
    readCSV = csv.reader(csvfile, delimiter='\n')
    for row in readCSV:
        vals = row[0].split(',')
        this_filepath = vals[0]
        image_names.append(this_filepath)
        
## remove duplicates by taking a set
image_names = list(set(image_names))
print(len(image_names))


321


In [6]:
print(image_names[:2])

['GrandJason_SEBlob_Nov2019_New_transparent_mosaic_group1_500x500_overlap40---598.png', 'SteepleJason_Bubble_Nov2019_Reprocess_transparent_mosaic_group1_500x500_overlap40---1119.png']


In [7]:
score_threshold_retinanet = 0.95
acceptable_box_overlap = 0.5

In [8]:
def get_gt_annotations(filepath):
    gt_ann = []
    with open (os.path.join(cwd, annotations_file)) as csvfile:
        readCSV = csv.reader(csvfile, delimiter='\n')
        for row in readCSV:
            vals = row[0].split(',')
            this_filepath = vals[0]
            
            if this_filepath == filepath:
                record = []
                record.append(int(vals[1]))
                record.append(int(vals[2]))
                record.append(int(vals[3]))
                record.append(int(vals[4]))
                record.append(vals[5])
                
                gt_ann.append(record)
    return gt_ann

In [9]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(int(boxA[0]), int(boxB[0]))
    yA = max(int(boxA[1]), int(boxB[1]))
    xB = min(int(boxA[2]), int(boxB[2]))
    yB = min(int(boxA[3]), int(boxB[3]))
 
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
 
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (int(boxA[2]) - int(boxA[0]) + 1) * (int(boxA[3]) - int(boxA[1]) + 1)
    boxBArea = (int(boxB[2]) - int(boxB[0]) + 1) * (int(boxB[3]) - int(boxB[1]) + 1)
 
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
 
    # return the intersection over union value
    return iou

In [10]:
def highest_iou(predicted_box, gt_ann):
    
    highest_iou = 0
    best_match = []
    for each_gt in gt_ann:
        this_iou = bb_intersection_over_union(predicted_box[:-1], each_gt[:-1])
#         print(" For this box is: ", this_iou)
        if this_iou > highest_iou:
            highest_iou = this_iou
            best_match = each_gt
            
    return highest_iou, best_match

In [11]:
def accuracy(gt_ann, predicted_ann):
    true_positive = []
    false_positive = []
    false_negative = []
    class_mismatch = []
    
    predicted_ann_copy = predicted_ann.copy()
    gt_ann_copy = gt_ann.copy()
    
    for each_pred in predicted_ann_copy:
        # 1. Calculate the highest_iou with any gt_box
        best_iou, best_gt_match = highest_iou(each_pred, gt_ann_copy)
#         print("Best IOU is: ", each_pred, best_gt_match, best_iou)
        
        ## If this box has a match
        if best_iou >= acceptable_box_overlap:
            ## if class label matches
            if best_gt_match[-1] == each_pred[-1]:
                ## This is a true positive
                true_positive.append(each_pred)
                ## remove this from predicted ann and gt
#                 print("Predicted Ann before:", predicted_ann)
                predicted_ann.remove(each_pred)
#                 print("Predicted Ann after:", predicted_ann)
                if best_gt_match in gt_ann:
                    gt_ann.remove(best_gt_match)
            elif best_gt_match[-1] != each_pred[-1]:
                ## this is a class mismatch
                class_mismatch.append(each_pred)
                ## remove this box
                predicted_ann.remove(each_pred)
                if best_gt_match in gt_ann:
                    gt_ann.remove(best_gt_match)
        
    ## If IOU is less than 0.5, leave as is

    ## Any predicted box is now false positive
    for remain_pred in predicted_ann:
        false_positive.append(remain_pred)

    ## Any remaining gt box is false negative
    for remain_gt in gt_ann:
        if remain_gt[-1] in ['penguin']:
            false_negative.append(remain_gt)

    return true_positive, false_positive, false_negative

In [12]:
def run_detection_image(filepath):
    image = read_image_bgr(filepath)

    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    print("processing time: ", time.time() - start)

    # correct for image scale
    boxes /= scale
    
    gt_ann = get_gt_annotations(filepath)
    num_gt_ann = len(gt_ann)
    
    predicted_ann = []

    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < score_threshold_retinanet:
            break         
        record = []
        record.append(int(box[0]))
        record.append(int(box[1]))
        record.append(int(box[2]))
        record.append(int(box[3]))
        record.append(labels_to_names[label])
        print("Record is: ", record)
        predicted_ann.append(record)
    
    
#     print("predicted ann are: ", predicted_ann)
    ## Calculate boxes
    true_positive, false_positive, false_negative = accuracy(gt_ann, predicted_ann)
    print(len(true_positive), len(false_positive), len(false_negative))
    
    #font = cv2.FONT_HERSHEY_SIMPLEX
    ## Draw theses on the image
    ## Draw true positive in green
    #if len(true_positive) > 0:
        #for each_true in true_positive:
            #cv2.rectangle(draw,(each_true[0],each_true[1]),(each_true[2],each_true[3]),(0,255,0),3) #green
            #cv2.putText(draw, each_true[-1], (each_true[0]-2, each_true[1]-2),font, 0.5,
                        #(0,0,0),1,cv2.LINE_AA) # text in black

    
     ## Draw false positive  in blue 
    #if len(false_positive) > 0:
        #for each_fp in false_positive:
            #cv2.rectangle(draw,(each_fp[0],each_fp[1]),(each_fp[2],each_fp[3]),(255,0,0),3) #green
            #cv2.putText(draw, each_fp[-1], (each_fp[0]-2, each_fp[1]-2),font, 0.5,
                        #(0,0,0),1,cv2.LINE_AA) # text in black
    
    
     ## Draw false negative in red 
    #if len(false_negative) > 0:
        #for each_fn in false_negative:
            #cv2.rectangle(draw,(each_fn[0],each_fn[1]),(each_fn[2],each_fn[3]),(0,0,255),3) #green
            #cv2.putText(draw, each_fn[-1], (each_fn[0]-2, each_fn[1]-2),font, 0.5,
                        #(0,0,0),1,cv2.LINE_AA) # text in black
            
            
    ## Add key to the image
    #cv2.putText(draw, "True Positive", (1200, 20),font, 0.8, (0,255,0),1,cv2.LINE_AA) 
    #cv2.putText(draw, "False Positive", (1200, 80),font, 0.8, (255,0,0),1,cv2.LINE_AA)
    #cv2.putText(draw, "False Negtaive", (1200, 110),font, 0.8, (0,0,2550),1,cv2.LINE_AA)
    
             
    ### Save this image
    
    #file, ext = os.path.splitext(filepath)
    #image_name = file.split('/')[-1] + ext
    #output_path = os.path.join('examples/results_test/', image_name)
    
    #draw_conv = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
    #cv2.imwrite(output_path, draw_conv)
    
    #plt.figure(figsize=(10, 10))
    #plt.axis('off')
    #plt.imshow(draw)
    #plt.show()
    
    return len(true_positive), len(false_positive), len(false_negative), num_gt_ann
        

In [13]:
total_true_positive = 0
total_false_positive = 0
total_false_negative = 0
total_gt = 0

In [14]:
for filepath in image_names:
    num_tp, num_fp, num_fn, num_gt = run_detection_image(filepath)
    print("Ground Truth: ", num_gt)
    total_true_positive += num_tp
    total_false_positive += num_fp
    total_false_negative += num_fn
    total_gt += num_gt

processing time:  3.2148149013519287
0 0 7
Ground Truth:  7
processing time:  0.11818909645080566
0 0 15
Ground Truth:  15
processing time:  0.1177515983581543
0 0 2
Ground Truth:  2
processing time:  0.11521506309509277
0 0 9
Ground Truth:  9
processing time:  0.11699771881103516
0 0 6
Ground Truth:  6
processing time:  0.11559605598449707
0 0 4
Ground Truth:  4
processing time:  0.12046408653259277
0 0 7
Ground Truth:  7
processing time:  0.11222434043884277
0 0 7
Ground Truth:  7
processing time:  0.11671900749206543
0 0 9
Ground Truth:  9
processing time:  0.11972737312316895
0 0 3
Ground Truth:  3
processing time:  0.12903404235839844
0 0 4
Ground Truth:  4
processing time:  0.11900115013122559
0 0 6
Ground Truth:  6
processing time:  0.11546492576599121
0 0 10
Ground Truth:  10
processing time:  0.11944699287414551
0 0 9
Ground Truth:  9
processing time:  0.12093830108642578
0 0 9
Ground Truth:  9
processing time:  0.1193084716796875
0 0 1
Ground Truth:  1
processing time:  0.121

processing time:  0.11917352676391602
0 0 7
Ground Truth:  7
processing time:  0.12032628059387207
0 0 4
Ground Truth:  4
processing time:  0.1215825080871582
0 0 10
Ground Truth:  10
processing time:  0.12061095237731934
0 0 5
Ground Truth:  5
processing time:  0.13497304916381836
0 0 6
Ground Truth:  6
processing time:  0.12221741676330566
0 0 1
Ground Truth:  1
processing time:  0.12614774703979492
0 0 8
Ground Truth:  8
processing time:  0.13432741165161133
0 0 4
Ground Truth:  4
processing time:  0.12268495559692383
0 0 13
Ground Truth:  13
processing time:  0.11899948120117188
0 0 3
Ground Truth:  3
processing time:  0.1312263011932373
0 0 14
Ground Truth:  14
processing time:  0.12119293212890625
0 0 2
Ground Truth:  2
processing time:  0.12074732780456543
0 0 3
Ground Truth:  3
processing time:  0.12678813934326172
0 0 2
Ground Truth:  2
processing time:  0.1323871612548828
0 0 9
Ground Truth:  9
processing time:  0.11873984336853027
0 0 6
Ground Truth:  6
processing time:  0.1

processing time:  0.12241482734680176
0 0 12
Ground Truth:  12
processing time:  0.12587594985961914
0 0 6
Ground Truth:  6
processing time:  0.1215369701385498
0 0 10
Ground Truth:  10
processing time:  0.11865854263305664
0 0 6
Ground Truth:  6
processing time:  0.1292259693145752
0 0 9
Ground Truth:  9
processing time:  0.12548303604125977
0 0 7
Ground Truth:  7
processing time:  0.13030266761779785
0 0 7
Ground Truth:  7
processing time:  0.1344585418701172
0 0 17
Ground Truth:  17
processing time:  0.1440584659576416
0 0 13
Ground Truth:  13
processing time:  0.12626099586486816
0 0 3
Ground Truth:  3
processing time:  0.1405651569366455
0 0 16
Ground Truth:  16
processing time:  0.1211550235748291
0 0 2
Ground Truth:  2
processing time:  0.1386411190032959
0 0 14
Ground Truth:  14
processing time:  0.12127137184143066
0 0 3
Ground Truth:  3
processing time:  0.1293935775756836
0 0 13
Ground Truth:  13
processing time:  0.11962413787841797
0 0 6
Ground Truth:  6
processing time:  

In [15]:
print(total_true_positive, total_false_positive, total_false_negative, total_gt)

0 0 2720 2720


In [16]:
precision = total_true_positive/(total_true_positive+total_false_positive)
recall = total_true_positive/(total_true_positive+ total_false_negative)
f1_score = 2*(precision * recall)/(precision + recall)

print(precision, recall, f1_score)

ZeroDivisionError: division by zero